# SMPC Protocols Explanation 
In this notebook I'll will give an overview and a quick explanation over the different SMPC protocols that are currently implemented in PySyft. I'll also elaborate on when their applicability to different use-cases (namely different computations) and compare their performance compared to each other and compared to the non-encrypted computations. 


## Quick recap - What is SMPC encryption?
As a quick recap "SMPC" or "MPC" stands for **Multi-Party Computation** and constitutes a form of encryption which can be used for Machine Learning (i.e. it is possible to do calculations on encrypted data) leveraging a network of min. 2 different servers. These are being trusted of not collaborating with each other and as a whole function as a **trusted execution environment** on which sensitive calculations such as model inference, training, etc. can be done without the model or the data being disclosed to any party besides the owner. 

## General concepts 
* Explain in brief important concepts that all protocols leverage 
  * crypto_store 
  * ...

---
--> START WITH LESS PERFORMANT PROTOCOL

In [ ]:
# Setup of example use-case problem 
# Serves to compare best models that can still be encrypted using the different protocols +
# Comparison to non-encrypted computation. (Only inference, we use pre-trained models)

## SecureNN Protocol
* Paper + Code 
* Intro to Garbled Circuits (or even earlier)
---
* **Structure:**
  * Theoretical Concept: Procedure, Safety Guarantees (Practical examples)
  * Theoretical Capabilities in terms of what computations are possible in encrypted form
  * *Concerning PySyft Implementation:* 
    * What was implemented, how? 
    * **MOST IMPORTANT / KEY QUESTION:** For what kind of computations, i.e. models, and in what kind of scenarios can I use this, i.e. what privacy and safety guarantees does this protocol provide in the current implementation. 
---
* **Code:**
  * Try to find most advanced model compatible encryptable with this approach
  * Do simple evaluation of most important metrics (possibly comparing with and without encryption)
    * Time 
    * ...

Important Concepts: 
--> Source: [The three part blog series about SPDZ](https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html) and [Morten Dahl's Blogs on SPDZ](https://mortendahl.github.io/2017/09/03/the-spdz-protocol-part1/)
* Functions are often refered to as circuits -> every function can be approx. represented by nested boolean circuits (combination of boolean gates) ??
* There are two types of SMPC: secret-sharing-based SMPC and circuit-gabling-based SMPC 
    * Basic protocol steps for secret-sharing-based SMPC see BLOG: https://bristolcrypto.blogspot.com/2016/10/what-is-spdz-part-1-mpc-circuit.html
    * Secret-sharing is mainly done via so called "Additive Secret Sharing" protocol, **Shamir's protocol** is a more complicated secret-sharing technique that doesn't only depend on additive splitting (reconstructable with a simple sum)
* SPDZ is a prominent example of secret-sharing-based SMPC 
  * It is considered to be the optimal method to use FHE in MPC according to authors of the original paper - as described in the blog(for the preprocessing)
  * The online phase is very good already (**TODO:** enumerate number of communications based on blog)
  * That's why mainly the offline phase - i.a. the preprocessing, generation of beaver triples and the big random numbers to serve to secret share values (the party having the real value also participates, other protocol than so far in tutorials where two dedicated units were used) - is being optimized. 
  * In the original SPDZ the preprocessing is done using Somewhat Homomorphic Encryption (SHE), for now no method exists to use instead FHE (**TODO:** How exactly? Are triples different for different functions?) in a competitive manner compared to other MPC protocols. 
  


## Functional Secret Sharing Protocol

### ARIANN Implementation 
* Paper + Code
---
* **Structure:**
  * Theoretical Concept: Procedure, Safety Guarantees (Practical examples)
  * Theoretical Capabilities in terms of what computations are possible in encrypted form
  * *Concerning PySyft Implementation:* 
    * What was implemented, how? 
    * **MOST IMPORTANT / KEY QUESTION:** For what kind of computations, i.e. models, and in what kind of scenarios can I use this, i.e. what privacy and safety guarantees does this protocol provide in the current implementation. 
---
* **Code:**
  * Try to find most advanced model compatible encryptable with this approach
  * Do simple evaluation of most important metrics (possibly comparing with and without encryption)
    * Time 
    * ...

# Other encryption protocols in PySyft 

## SPDZ Protocol 
* Paper + Code
* Introduction of first protocol along quick intro into beaver triples 

## FALCON Protocol (Future)
* Paper + Code 